In [1]:
# Import data manipulation libraries
import pandas as pd
import numpy as np
import math

# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Import statistical and data processing functions
from scipy.stats import norm, skew
from sklearn.preprocessing import StandardScaler

# Import machine learning models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Import model evaluation tools
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score, train_test_split

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('/home/chelsea-castro/virtual-machine/notebook/train.csv')
df.head(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [5]:
# Step 1: Drop high-missing-value columns and redundant identifiers
columns_to_drop = [
    'Alley', 'PoolQC', 'Fence', 'MiscFeature',  # High missing values
    'Id', 'Utilities', 'Street'  # Redundant columns
]
df_cleaned = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Step 2: Create aggregate features

# 2.1 Total square footage (assuming all necessary columns are present)
df_cleaned['TotalSF'] = df['1stFlrSF'] + df['2ndFlrSF'] + df['LowQualFinSF'] + df['TotalBsmtSF']

# 2.2 Total porch square footage
df_cleaned['TotalPorchSF'] = (
    df['WoodDeckSF'] + df['OpenPorchSF'] + df['EnclosedPorch'] + df['3SsnPorch'] + df['ScreenPorch']
)

# 2.3 Garage-related: Garage capacity (average space per car) and age of the garage
if 'GarageArea' in df.columns and 'GarageCars' in df.columns:
    df_cleaned['GarageCapacity'] = df['GarageArea'] / df['GarageCars']

if 'YrSold' in df.columns and 'GarageYrBlt' in df.columns:
    df_cleaned['GarageAge'] = df['YrSold'] - df['GarageYrBlt']

# 2.4 Total basement square footage
if all(col in df.columns for col in ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF']):
    df_cleaned['TotalBsmtSF'] = df['BsmtFinSF1'] + df['BsmtFinSF2'] + df['BsmtUnfSF']

# 2.5 Total bathrooms count
df_cleaned['TotalBath'] = df['FullBath'] + (0.5 * df['HalfBath'])

# Step 3: Age transformations

# 3.1 House age and Remodel age based on year sold
if 'YrSold' in df.columns and 'YearBuilt' in df.columns:
    df_cleaned['HouseAge'] = df['YrSold'] - df['YearBuilt']
if 'YrSold' in df.columns and 'YearRemodAdd' in df.columns:
    df_cleaned['RemodelAge'] = df['YrSold'] - df['YearRemodAdd']

# Step 4: Functional transformation (binary)
df_cleaned['FunctionalTypical'] = df['Functional'].apply(lambda x: 1 if x == 'Typ' else 0)

# Step 5: Drop original columns used in aggregation
columns_to_drop_after_merge = [
    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'TotalBsmtSF', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'GarageArea', 'GarageCars', 'GarageYrBlt', 'BsmtFinSF1', 'BsmtFinSF2',
    'BsmtUnfSF', 'FullBath', 'HalfBath', 'YearBuilt', 'YearRemodAdd', 'Functional', 'MoSold', 'YrSold'
]
df_final = df_cleaned.drop(columns=[col for col in columns_to_drop_after_merge if col in df_cleaned.columns])

# Display the cleaned DataFrame's columns to verify
df_final.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,GrLivArea,BsmtFullBath,BsmtHalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolArea,MiscVal,SaleType,SaleCondition,SalePrice,TotalSF,TotalPorchSF,GarageCapacity,GarageAge,TotalBath,HouseAge,RemodelAge,FunctionalTypical
0,60,RL,65.0,8450,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,1710,1,0,3,1,Gd,8,0,NaN,Attchd,RFn,TA,TA,Y,0,0,WD,Normal,208500,2566,61,274.000000,5.0,2.5,5,5,1
1,20,RL,80.0,9600,Reg,Lvl,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,1262,0,1,3,1,TA,6,1,TA,Attchd,RFn,TA,TA,Y,0,0,WD,Normal,181500,2524,298,230.000000,31.0,2.0,31,31,1
2,60,RL,68.0,11250,IR1,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,1786,1,0,3,1,Gd,6,1,TA,Attchd,RFn,TA,TA,Y,0,0,WD,Normal,223500,2706,42,304.000000,7.0,2.5,7,6,1
3,70,RL,60.0,9550,IR1,Lvl,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,1717,1,0,3,1,Gd,7,1,Gd,Detchd,Unf,TA,TA,Y,0,0,WD,Abnorml,140000,2473,307,214.000000,8.0,1.0,91,36,1
4,60,RL,84.0,14260,IR1,Lvl,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,2198,1,0,4,1,Gd,9,1,TA,Attchd,RFn,TA,TA,Y,0,0,WD,Normal,250000,3343,276,278.666667,8.0,2.5,8,8,1


In [6]:
# Step 1: Create 'GarageCapacity' and 'GarageAge' if the necessary columns are present
# 1.1 GarageCapacity: Calculate average space per car
if 'GarageArea' in df_final.columns and 'GarageCars' in df_final.columns:
    df_final['GarageCapacity'] = df_final['GarageArea'] / df_final['GarageCars']

# 1.2 GarageAge: Calculate age of the garage at the time of sale
if 'YrSold' in df_final.columns and 'GarageYrBlt' in df_final.columns:
    df_final['GarageAge'] = df_final['YrSold'] - df_final['GarageYrBlt']

# Step 2: Fill missing values based on analysis

# 2.1 LotFrontage - Fill with median LotFrontage by Neighborhood
if 'LotFrontage' in df_final.columns and 'Neighborhood' in df_final.columns:
    df_final['LotFrontage'] = df_final.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# 2.2 Masonry Veneer (MasVnrType and MasVnrArea)
if 'MasVnrType' in df_final.columns:
    df_final['MasVnrType'].fillna('None', inplace=True)
if 'MasVnrArea' in df_final.columns:
    df_final['MasVnrArea'].fillna(0, inplace=True)

# 2.3 Basement-related Columns - Set missing values to "No Basement" or 0 for homes without basements
basement_columns = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
for col in basement_columns:
    if col in df_final.columns:
        df_final[col].fillna('No Basement', inplace=True)

# 2.4 Electrical - Fill with the mode as there's only 1 missing value
if 'Electrical' in df_final.columns:
    df_final['Electrical'].fillna(df_final['Electrical'].mode()[0], inplace=True)

# 2.5 FireplaceQu - Set to "No Fireplace" for missing values
if 'FireplaceQu' in df_final.columns:
    df_final['FireplaceQu'].fillna('No Fireplace', inplace=True)

# 2.6 Garage-related Columns - Fill missing values with "No Garage" or 0 where appropriate
garage_related_cols = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
for col in garage_related_cols:
    if col in df_final.columns:
        df_final[col].fillna('No Garage', inplace=True)

# Fill 'GarageCapacity' and 'GarageAge' with 0 where missing, now that they have been created
if 'GarageCapacity' in df_final.columns:
    df_final['GarageCapacity'].fillna(0, inplace=True)
if 'GarageAge' in df_final.columns:
    df_final['GarageAge'].fillna(0, inplace=True)

# Drop the 'Alley' column due to high percentage of missing values
if 'Alley' in df_final.columns:
    df_final.drop(columns=['Alley'], inplace=True)

df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 60 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MSSubClass         1460 non-null   int64  
 1   MSZoning           1460 non-null   object 
 2   LotFrontage        1460 non-null   float64
 3   LotArea            1460 non-null   int64  
 4   LotShape           1460 non-null   object 
 5   LandContour        1460 non-null   object 
 6   LotConfig          1460 non-null   object 
 7   LandSlope          1460 non-null   object 
 8   Neighborhood       1460 non-null   object 
 9   Condition1         1460 non-null   object 
 10  Condition2         1460 non-null   object 
 11  BldgType           1460 non-null   object 
 12  HouseStyle         1460 non-null   object 
 13  OverallQual        1460 non-null   int64  
 14  OverallCond        1460 non-null   int64  
 15  RoofStyle          1460 non-null   object 
 16  RoofMatl           1460 

In [7]:
# Proceeding with feature engineering directly on the dataset based on available columns

# 1. Total Square Footage (TotalSF): Adding 'GrLivArea' and 'TotalBsmtSF' if both are present
if all(col in df_final.columns for col in ['GrLivArea', 'TotalBsmtSF']):
    df_final['TotalSF'] = df_final['GrLivArea'] + df_final['TotalBsmtSF']

# 2. Total Porch Square Footage (TotalPorchSF): Sum of all porch-related areas
porch_columns = ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
if all(col in df_final.columns for col in porch_columns):
    df_final['TotalPorchSF'] = df_final[porch_columns].sum(axis=1)

# 3. Total Bathrooms (TotalBath): Sum of full and half baths, including basement baths
bath_columns = ['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath']
if all(col in df_final.columns for col in bath_columns):
    df_final['TotalBath'] = (
        df_final['FullBath'] + (df_final['HalfBath'] * 0.5) +
        df_final['BsmtFullBath'] + (df_final['BsmtHalfBath'] * 0.5)
    )

# 4. House Age and Remodel Age at time of sale
if all(col in df_final.columns for col in ['YrSold', 'YearBuilt']):
    df_final['HouseAge'] = df_final['YrSold'] - df_final['YearBuilt']
if all(col in df_final.columns for col in ['YrSold', 'YearRemodAdd']):
    df_final['RemodelAge'] = df_final['YrSold'] - df_final['YearRemodAdd']

# 5. FunctionalTypical: Binary indicator if functionality is typical (Typ)
if 'Functional' in df_final.columns:
    df_final['FunctionalTypical'] = df_final['Functional'].apply(lambda x: 1 if x == 'Typ' else 0)

# Display the resulting DataFrame with new features
df_final.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,GrLivArea,BsmtFullBath,BsmtHalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolArea,MiscVal,SaleType,SaleCondition,SalePrice,TotalSF,TotalPorchSF,GarageCapacity,GarageAge,TotalBath,HouseAge,RemodelAge,FunctionalTypical
0,60,RL,65.0,8450,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,1710,1,0,3,1,Gd,8,0,No Fireplace,Attchd,RFn,TA,TA,Y,0,0,WD,Normal,208500,2566,61,274.000000,5.0,2.5,5,5,1
1,20,RL,80.0,9600,Reg,Lvl,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,1262,0,1,3,1,TA,6,1,TA,Attchd,RFn,TA,TA,Y,0,0,WD,Normal,181500,2524,298,230.000000,31.0,2.0,31,31,1
2,60,RL,68.0,11250,IR1,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,1786,1,0,3,1,Gd,6,1,TA,Attchd,RFn,TA,TA,Y,0,0,WD,Normal,223500,2706,42,304.000000,7.0,2.5,7,6,1
3,70,RL,60.0,9550,IR1,Lvl,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,1717,1,0,3,1,Gd,7,1,Gd,Detchd,Unf,TA,TA,Y,0,0,WD,Abnorml,140000,2473,307,214.000000,8.0,1.0,91,36,1
4,60,RL,84.0,14260,IR1,Lvl,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,2198,1,0,4,1,Gd,9,1,TA,Attchd,RFn,TA,TA,Y,0,0,WD,Normal,250000,3343,276,278.666667,8.0,2.5,8,8,1


In [8]:
# Implementing additional merges and removals

# 1. Creating TotalRooms by combining rooms above grade and bedrooms
if 'TotRmsAbvGrd' in df_final.columns and 'BedroomAbvGr' in df_final.columns:
    df_final['TotalRooms'] = df_final['TotRmsAbvGrd'] + df_final['BedroomAbvGr']

# 2. Calculating BedBathRatio (ratio of bedrooms to total baths)
if 'BedroomAbvGr' in df_final.columns and 'TotalBath' in df_final.columns:
    df_final['BedBathRatio'] = df_final['BedroomAbvGr'] / df_final['TotalBath'].replace(0, 0.1)

# 3. Creating TotalParkingSpace from GarageCapacity and PavedDrive
if 'GarageCapacity' in df_final.columns and 'PavedDrive' in df_final.columns:
    df_final['TotalParkingSpace'] = df_final['GarageCapacity'] + df_final['PavedDrive'].apply(lambda x: 1 if x == 'Y' else 0)

# 4. Converting HouseAge and RemodelAge into categorical age groups
def age_category(age):
    if age < 10:
        return "New"
    elif age < 30:
        return "Moderate"
    else:
        return "Old"

if 'HouseAge' in df_final.columns:
    df_final['HouseAgeGroup'] = df_final['HouseAge'].apply(age_category)

if 'RemodelAge' in df_final.columns:
    df_final['RemodelAgeGroup'] = df_final['RemodelAge'].apply(age_category)

# 5. Dropping year-related columns
year_related_columns = ['YrSold', 'YearBuilt', 'YearRemodAdd']
df_final.drop(columns=[col for col in year_related_columns if col in df_final.columns], inplace=True)

# 6. Dropping Utilities and Street
low_variability_columns = ['Utilities', 'Street']
df_final.drop(columns=[col for col in low_variability_columns if col in df_final.columns], inplace=True)

# 7. Dropping OverallCond and OverallQual
quality_columns = ['OverallCond', 'OverallQual']
df_final.drop(columns=[col for col in quality_columns if col in df_final.columns], inplace=True)

df_final.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,GrLivArea,BsmtFullBath,BsmtHalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolArea,MiscVal,SaleType,SaleCondition,SalePrice,TotalSF,TotalPorchSF,GarageCapacity,GarageAge,TotalBath,HouseAge,RemodelAge,FunctionalTypical,TotalRooms,BedBathRatio,TotalParkingSpace,HouseAgeGroup,RemodelAgeGroup
0,60,RL,65.0,8450,Reg,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,1710,1,0,3,1,Gd,8,0,No Fireplace,Attchd,RFn,TA,TA,Y,0,0,WD,Normal,208500,2566,61,274.000000,5.0,2.5,5,5,1,11,1.2,275.000000,New,New
1,20,RL,80.0,9600,Reg,Lvl,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,1262,0,1,3,1,TA,6,1,TA,Attchd,RFn,TA,TA,Y,0,0,WD,Normal,181500,2524,298,230.000000,31.0,2.0,31,31,1,9,1.5,231.000000,Old,Old
2,60,RL,68.0,11250,IR1,Lvl,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,1786,1,0,3,1,Gd,6,1,TA,Attchd,RFn,TA,TA,Y,0,0,WD,Normal,223500,2706,42,304.000000,7.0,2.5,7,6,1,9,1.2,305.000000,New,New
3,70,RL,60.0,9550,IR1,Lvl,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,1717,1,0,3,1,Gd,7,1,Gd,Detchd,Unf,TA,TA,Y,0,0,WD,Abnorml,140000,2473,307,214.000000,8.0,1.0,91,36,1,10,3.0,215.000000,Old,Old
4,60,RL,84.0,14260,IR1,Lvl,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,2198,1,0,4,1,Gd,9,1,TA,Attchd,RFn,TA,TA,Y,0,0,WD,Normal,250000,3343,276,278.666667,8.0,2.5,8,8,1,13,1.6,279.666667,New,New


In [9]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 63 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   MSSubClass         1460 non-null   int64  
 1   MSZoning           1460 non-null   object 
 2   LotFrontage        1460 non-null   float64
 3   LotArea            1460 non-null   int64  
 4   LotShape           1460 non-null   object 
 5   LandContour        1460 non-null   object 
 6   LotConfig          1460 non-null   object 
 7   LandSlope          1460 non-null   object 
 8   Neighborhood       1460 non-null   object 
 9   Condition1         1460 non-null   object 
 10  Condition2         1460 non-null   object 
 11  BldgType           1460 non-null   object 
 12  HouseStyle         1460 non-null   object 
 13  RoofStyle          1460 non-null   object 
 14  RoofMatl           1460 non-null   object 
 15  Exterior1st        1460 non-null   object 
 16  Exterior2nd        1460 

In [10]:
# Combine GarageQual and GarageCond into GarageOverall if they are categorical or ordinal
if 'GarageQual' in df_final.columns and 'GarageCond' in df_final.columns:
    df_final['GarageOverall'] = df_final[['GarageQual', 'GarageCond']].apply(lambda x: x.mode()[0], axis=1)

# Optionally drop GarageType if not needed for prediction
if 'GarageType' in df_final.columns:
    df_final.drop(columns=['GarageType'], inplace=True)

# Combine BsmtQual and BsmtCond into BsmtOverall (assuming categorical or ordinal)
if 'BsmtQual' in df_final.columns and 'BsmtCond' in df_final.columns:
    df_final['BsmtOverall'] = df_final[['BsmtQual', 'BsmtCond']].apply(lambda x: x.mode()[0], axis=1)

# Merge BsmtFinType1 and BsmtFinType2 into a single score or label for TotalBsmtFinType
fin_type_mapping = {'GLQ': 5, 'ALQ': 4, 'BLQ': 3, 'Rec': 2, 'LwQ': 1, 'Unf': 0, 'NA': 0}
if 'BsmtFinType1' in df_final.columns and 'BsmtFinType2' in df_final.columns:
    df_final['TotalBsmtFinType'] = df_final['BsmtFinType1'].map(fin_type_mapping) + df_final['BsmtFinType2'].map(fin_type_mapping)

# Drop the now-redundant columns after merging
redundant_columns = ['GarageQual', 'GarageCond', 'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2']
df_final.drop(columns=[col for col in redundant_columns if col in df_final.columns], inplace=True)

# 1. ConditionScore: Combining Condition1 and Condition2 to create a single condition metric
if 'Condition1' in df_final.columns and 'Condition2' in df_final.columns:
    df_final['ConditionScore'] = df_final[['Condition1', 'Condition2']].apply(lambda x: x.mode()[0], axis=1)
    df_final.drop(columns=['Condition1', 'Condition2'], inplace=True)

# 2. ExteriorOverall: Combining Exterior1st and Exterior2nd
if 'Exterior1st' in df_final.columns and 'Exterior2nd' in df_final.columns:
    df_final['ExteriorOverall'] = df_final[['Exterior1st', 'Exterior2nd']].apply(lambda x: x.mode()[0], axis=1)
    df_final.drop(columns=['Exterior1st', 'Exterior2nd'], inplace=True)

# 3. PavedDrive: Convert to binary or remove if redundant
if 'PavedDrive' in df_final.columns:
    df_final['PavedDriveBinary'] = df_final['PavedDrive'].apply(lambda x: 1 if x == 'Y' else 0)
    df_final.drop(columns=['PavedDrive'], inplace=True)

# 4. HeatingQuality: Merging Heating and HeatingQC into one feature if both exist
if 'Heating' in df_final.columns and 'HeatingQC' in df_final.columns:
    df_final['HeatingQuality'] = df_final[['Heating', 'HeatingQC']].apply(lambda x: x.mode()[0], axis=1)
    df_final.drop(columns=['Heating'], inplace=True)

# 5. PoolScore: Merging PoolQC and PoolArea
if 'PoolArea' in df_final.columns and 'PoolQC' in df_final.columns:
    pool_quality_mapping = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'NA': 0}
    df_final['PoolScore'] = df_final['PoolArea'] * df_final['PoolQC'].map(pool_quality_mapping).fillna(0)
    df_final.drop(columns=['PoolArea', 'PoolQC'], inplace=True)

# 6. MiscFeatureBinary: Convert MiscVal to binary if necessary
if 'MiscVal' in df_final.columns:
    df_final['MiscFeatureBinary'] = df_final['MiscVal'].apply(lambda x: 1 if x > 0 else 0)
    df_final.drop(columns=['MiscVal'], inplace=True)

# 7. Binning Neighborhood and Quality/Condition
# Assuming Neighborhood and OverallQual/OverallCond still exist
if 'Neighborhood' in df_final.columns:
    high_value_neighborhoods = ['NridgHt', 'NoRidge', 'StoneBr']  # Example high-value neighborhoods
    df_final['NeighborhoodBinned'] = df_final['Neighborhood'].apply(lambda x: 'High' if x in high_value_neighborhoods else 'Other')
    df_final.drop(columns=['Neighborhood'], inplace=True)

# Binning OverallQual and OverallCond if they still exist
for col in ['OverallQual', 'OverallCond']:
    if col in df_final.columns:
        df_final[col + '_Binned'] = pd.cut(df_final[col], bins=[0, 4, 6, 10], labels=['Low', 'Medium', 'High'])
        df_final.drop(columns=[col], inplace=True)

df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 57 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MSSubClass          1460 non-null   int64  
 1   MSZoning            1460 non-null   object 
 2   LotFrontage         1460 non-null   float64
 3   LotArea             1460 non-null   int64  
 4   LotShape            1460 non-null   object 
 5   LandContour         1460 non-null   object 
 6   LotConfig           1460 non-null   object 
 7   LandSlope           1460 non-null   object 
 8   BldgType            1460 non-null   object 
 9   HouseStyle          1460 non-null   object 
 10  RoofStyle           1460 non-null   object 
 11  RoofMatl            1460 non-null   object 
 12  MasVnrType          1460 non-null   object 
 13  MasVnrArea          1460 non-null   float64
 14  ExterQual           1460 non-null   object 
 15  ExterCond           1460 non-null   object 
 16  Founda

In [11]:
# Dropping features based on the rationale provided
features_to_drop = [
    'HouseAgeGroup', 'RemodelAgeGroup', 'ConditionScore', 'ExteriorOverall',
    'FunctionalTypical', 'HeatingQuality', 'FireplaceQu', 'GarageOverall', 'BsmtOverall',
    'PavedDriveBinary', 'CentralAir', 'TotRmsAbvGrd', 'MiscFeatureBinary', 'BedBathRatio'
]

df_final.drop(columns=[col for col in features_to_drop if col in df_final.columns], inplace=True)

# Display the final cleaned dataset to verify the reduced number of features
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MSSubClass          1460 non-null   int64  
 1   MSZoning            1460 non-null   object 
 2   LotFrontage         1460 non-null   float64
 3   LotArea             1460 non-null   int64  
 4   LotShape            1460 non-null   object 
 5   LandContour         1460 non-null   object 
 6   LotConfig           1460 non-null   object 
 7   LandSlope           1460 non-null   object 
 8   BldgType            1460 non-null   object 
 9   HouseStyle          1460 non-null   object 
 10  RoofStyle           1460 non-null   object 
 11  RoofMatl            1460 non-null   object 
 12  MasVnrType          1460 non-null   object 
 13  MasVnrArea          1460 non-null   float64
 14  ExterQual           1460 non-null   object 
 15  ExterCond           1460 non-null   object 
 16  Founda

In [12]:
# Dropping additional low-impact or redundant features
final_features_to_drop = [
    'LotShape', 'LandContour', 'RoofStyle', 'RoofMatl', 'Foundation',
    'PoolArea', 'BsmtFullBath', 'BsmtHalfBath', 'Electrical']

df_final.drop(columns=[col for col in final_features_to_drop if col in df_final.columns], inplace=True)

df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MSSubClass          1460 non-null   int64  
 1   MSZoning            1460 non-null   object 
 2   LotFrontage         1460 non-null   float64
 3   LotArea             1460 non-null   int64  
 4   LotConfig           1460 non-null   object 
 5   LandSlope           1460 non-null   object 
 6   BldgType            1460 non-null   object 
 7   HouseStyle          1460 non-null   object 
 8   MasVnrType          1460 non-null   object 
 9   MasVnrArea          1460 non-null   float64
 10  ExterQual           1460 non-null   object 
 11  ExterCond           1460 non-null   object 
 12  BsmtExposure        1460 non-null   object 
 13  HeatingQC           1460 non-null   object 
 14  GrLivArea           1460 non-null   int64  
 15  BedroomAbvGr        1460 non-null   int64  
 16  Kitche

In [13]:
# Defining the rank mappings for each applicable column
rank_mappings = {
    'MSSubClass': {160: 1, 120: 2, 180: 3, 60: 4, 20: 5, 50: 6, 75: 7, 85: 8, 90: 9, 70: 10, 40: 11, 30: 12, 
                   45: 13, 150: 14, 190: 15, 80: 16},
    'MSZoning': {'FV': 1, 'RL': 2, 'RP': 3, 'RM': 4, 'RH': 5, 'C': 6, 'A': 7, 'I': 8},
    'LotConfig': {'CulDSac': 1, 'Corner': 2, 'FR2': 3, 'Inside': 4},
    'LandSlope': {'Gtl': 1, 'Mod': 2, 'Sev': 3},
    'BldgType': {'1Fam': 1, 'TwnhsE': 2, 'TwnhsI': 3, 'Duplx': 4, '2FmCon': 5},
    'HouseStyle': {'2.5Fin': 1, '2Story': 2, '1Story': 3, '1.5Fin': 4, 'SLvl': 5, 'SFoyer': 6, '1.5Unf': 7, '2.5Unf': 8},
    'MasVnrType': {'Stone': 1, 'BrkFace': 2, 'BrkCmn': 3, 'CBlock': 4, 'None': 5},
    'ExterQual': {'Ex': 1, 'Gd': 2, 'TA': 3, 'Fa': 4, 'Po': 5},
    'ExterCond': {'Ex': 1, 'Gd': 2, 'TA': 3, 'Fa': 4, 'Po': 5},
    'BsmtExposure': {'Gd': 1, 'Av': 2, 'Mn': 3, 'No': 4, 'NA': 5},
    'HeatingQC': {'Ex': 1, 'Gd': 2, 'TA': 3, 'Fa': 4, 'Po': 5},
    'KitchenQual': {'Ex': 1, 'Gd': 2, 'TA': 3, 'Fa': 4, 'Po': 5},
    'GarageFinish': {'Fin': 1, 'RFn': 2, 'Unf': 3, 'NA': 4},
    'SaleType': {'New': 1, 'WD': 2, 'CWD': 3, 'VWD': 4, 'Con': 5, 'ConLD': 6, 'ConLI': 7, 'ConLw': 8, 'COD': 9, 'Oth': 10},
    'SaleCondition': {'Normal': 1, 'Partial': 2, 'Alloca': 3, 'AdjLand': 4, 'Family': 5, 'Abnorml': 6},
}

# Applying the mappings to each column in df_final
for col, mapping in rank_mappings.items():
    if col in df_final.columns:
        df_final[col] = df_final[col].map(mapping)

# Display the updated DataFrame to verify
df_final.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,LotConfig,LandSlope,BldgType,HouseStyle,MasVnrType,MasVnrArea,ExterQual,ExterCond,BsmtExposure,HeatingQC,GrLivArea,BedroomAbvGr,KitchenAbvGr,KitchenQual,Fireplaces,GarageFinish,SaleType,SaleCondition,SalePrice,TotalSF,TotalPorchSF,GarageCapacity,GarageAge,TotalBath,HouseAge,RemodelAge,TotalRooms,TotalParkingSpace,TotalBsmtFinType,NeighborhoodBinned
0,4,2.0,65.0,8450,4.0,1,1.0,2,2,196.0,2,3,4.0,1,1710,3,1,2,0,2.0,2,1,208500,2566,61,274.000000,5.0,2.5,5,5,11,275.000000,5.0,Other
1,5,2.0,80.0,9600,3.0,1,1.0,3,5,0.0,3,3,1.0,1,1262,3,1,3,1,2.0,2,1,181500,2524,298,230.000000,31.0,2.0,31,31,9,231.000000,4.0,Other
2,4,2.0,68.0,11250,4.0,1,1.0,2,2,162.0,2,3,3.0,1,1786,3,1,2,1,2.0,2,1,223500,2706,42,304.000000,7.0,2.5,7,6,9,305.000000,5.0,Other
3,10,2.0,60.0,9550,2.0,1,1.0,2,5,0.0,3,3,4.0,2,1717,3,1,2,1,3.0,2,6,140000,2473,307,214.000000,8.0,1.0,91,36,10,215.000000,4.0,Other
4,4,2.0,84.0,14260,3.0,1,1.0,2,2,350.0,2,3,2.0,1,2198,4,1,2,1,2.0,2,1,250000,3343,276,278.666667,8.0,2.5,8,8,13,279.666667,5.0,High


In [15]:
df_final['NeighborhoodBinned'] = df_final['NeighborhoodBinned'].replace({'High': 1, 'Other': 0.5})

In [16]:
print(df_final['NeighborhoodBinned'].unique())

[0.5 1. ]


In [23]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MSSubClass          1460 non-null   int64  
 1   MSZoning            1450 non-null   float64
 2   LotFrontage         1460 non-null   float64
 3   LotArea             1460 non-null   int64  
 4   LotConfig           1456 non-null   float64
 5   LandSlope           1460 non-null   int64  
 6   BldgType            1334 non-null   float64
 7   HouseStyle          1460 non-null   int64  
 8   MasVnrType          1460 non-null   int64  
 9   MasVnrArea          1460 non-null   float64
 10  ExterQual           1460 non-null   int64  
 11  ExterCond           1460 non-null   int64  
 12  BsmtExposure        1422 non-null   float64
 13  HeatingQC           1460 non-null   int64  
 14  GrLivArea           1460 non-null   int64  
 15  BedroomAbvGr        1460 non-null   int64  
 16  Kitche

In [24]:
df_final.fillna(0, inplace=True)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   MSSubClass          1460 non-null   int64  
 1   MSZoning            1460 non-null   float64
 2   LotFrontage         1460 non-null   float64
 3   LotArea             1460 non-null   int64  
 4   LotConfig           1460 non-null   float64
 5   LandSlope           1460 non-null   int64  
 6   BldgType            1460 non-null   float64
 7   HouseStyle          1460 non-null   int64  
 8   MasVnrType          1460 non-null   int64  
 9   MasVnrArea          1460 non-null   float64
 10  ExterQual           1460 non-null   int64  
 11  ExterCond           1460 non-null   int64  
 12  BsmtExposure        1460 non-null   float64
 13  HeatingQC           1460 non-null   int64  
 14  GrLivArea           1460 non-null   int64  
 15  BedroomAbvGr        1460 non-null   int64  
 16  Kitche

In [20]:
!pip install scikit-optimize

In [25]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from skopt import BayesSearchCV  # For Bayesian Optimization

# Assuming 'df_final' is the dataset and 'SalePrice' is the target variable
X = df_final.drop(columns=['SalePrice'])
y = df_final['SalePrice']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# Initialize and train the linear regression model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predict on test set
y_pred = linear_model.predict(X_test)

# Print all evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Initial Linear Model - Mean Squared Error (MSE): {mse}")
print(f"Initial Linear Model - Mean Absolute Error (MAE): {mae}")
print(f"Initial Linear Model - R^2 Score: {r2}")

# Cross-validation for baseline comparison
cv_scores = cross_val_score(linear_model, X, y, cv=5, scoring='neg_mean_squared_error')
print(f"Cross-Validated MSE (Initial Model): {-cv_scores.mean()}")

Initial Linear Model - Mean Squared Error (MSE): 1165286529.061449
Initial Linear Model - Mean Absolute Error (MAE): 21929.27283074668
Initial Linear Model - R^2 Score: 0.8480786217449825
Cross-Validated MSE (Initial Model): 1281072385.530836


In [28]:
# Define parameter space for Bayesian optimization
param_space = {
    'fit_intercept': [True, False],
}

# Bayesian Optimization with Cross-Validation
bayes_opt = BayesSearchCV(
    estimator=LinearRegression(),
    search_spaces=param_space,
    n_iter=10,  # Number of iterations for the search
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Fit the Bayesian Optimization
bayes_opt.fit(X_train, y_train)

BayesSearchCV(cv=5, estimator=LinearRegression(), n_iter=10, random_state=42,
              scoring='neg_mean_squared_error',
              search_spaces={'fit_intercept': [True, False]})

In [29]:
# Retrieve and print the best parameters and best score from Bayesian optimization
print("Best Parameters from Bayesian Optimization:", bayes_opt.best_params_)
print("Best Cross-Validated MSE from Bayesian Optimization:", -bayes_opt.best_score_)

# Using the best estimator from Bayesian Optimization
optimized_model = bayes_opt.best_estimator_

# Predict with the optimized model
y_pred_optimized = optimized_model.predict(X_test)

Best Parameters from Bayesian Optimization: OrderedDict({'fit_intercept': True})
Best Cross-Validated MSE from Bayesian Optimization: 1367530340.9086916


In [30]:
# Calculate metrics for optimized model
mse_optimized = mean_squared_error(y_test, y_pred_optimized)
mae_optimized = mean_absolute_error(y_test, y_pred_optimized)
r2_optimized = r2_score(y_test, y_pred_optimized)

# Print metrics for comparison
print("\nComparison of Linear Regression Models:")
print("Initial Linear Regression Model:")
print(f"  MSE: {mse}, MAE: {mae}, R^2: {r2}")
print("Optimized Linear Regression Model:")
print(f"  MSE: {mse_optimized}, MAE: {mae_optimized}, R^2: {r2_optimized}")


Comparison of Linear Regression Models:
Initial Linear Regression Model:
  MSE: 1165286529.061449, MAE: 21929.27283074668, R^2: 0.8480786217449825
Optimized Linear Regression Model:
  MSE: 1165286529.061449, MAE: 21929.27283074668, R^2: 0.8480786217449825


In [31]:
! pip install xgboost scikit-optimize

In [32]:
# Initialize and train the XGBoost regression model
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)

# Predict on test set
y_pred_xgb = xgb_model.predict(X_test)

# Print all evaluation metrics
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"XGBoost Model - Mean Squared Error (MSE): {mse_xgb}")
print(f"XGBoost Model - Mean Absolute Error (MAE): {mae_xgb}")
print(f"XGBoost Model - R^2 Score: {r2_xgb}")

# Cross-validation for baseline comparison
cv_scores_xgb = cross_val_score(xgb_model, X, y, cv=5, scoring='neg_mean_squared_error')
print(f"Cross-Validated MSE (XGBoost Model): {-cv_scores_xgb.mean()}")

XGBoost Model - Mean Squared Error (MSE): 952030990.7683752
XGBoost Model - Mean Absolute Error (MAE): 19811.910878638697
XGBoost Model - R^2 Score: 0.8758813142776489
Cross-Validated MSE (XGBoost Model): 898537371.7970164


In [33]:
from skopt import BayesSearchCV

# Define parameter space for Bayesian optimization
param_space_xgb = {
    'n_estimators': (100, 1000),  # Number of boosting rounds
    'max_depth': (3, 10),          # Maximum tree depth
    'learning_rate': (0.01, 0.3, 'uniform'),  # Step size shrinkage
    'subsample': (0.5, 1.0),       # Fraction of samples used for training
}

# Bayesian Optimization with Cross-Validation for XGBoost
bayes_opt_xgb = BayesSearchCV(
    estimator=XGBRegressor(objective='reg:squarederror', random_state=42),
    search_spaces=param_space_xgb,
    n_iter=10,  # Number of iterations for the search
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Fit the Bayesian Optimization
bayes_opt_xgb.fit(X_train, y_train)

# Retrieve and print the best parameters and best score from Bayesian optimization
print("Best Parameters from Bayesian Optimization (XGBoost):", bayes_opt_xgb.best_params_)
print("Best Cross-Validated MSE from Bayesian Optimization (XGBoost):", -bayes_opt_xgb.best_score_)

Best Parameters from Bayesian Optimization (XGBoost): OrderedDict({'learning_rate': 0.011052789103875535, 'max_depth': 9, 'n_estimators': 767, 'subsample': 0.6272771289320225})
Best Cross-Validated MSE from Bayesian Optimization (XGBoost): 978061785.9396536


In [34]:
# Using the best estimator from Bayesian Optimization for XGBoost
optimized_xgb_model = bayes_opt_xgb.best_estimator_

# Predict with the optimized model
y_pred_optimized_xgb = optimized_xgb_model.predict(X_test)

# Calculate metrics for optimized model
mse_optimized_xgb = mean_squared_error(y_test, y_pred_optimized_xgb)
mae_optimized_xgb = mean_absolute_error(y_test, y_pred_optimized_xgb)
r2_optimized_xgb = r2_score(y_test, y_pred_optimized_xgb)

# Print metrics for comparison
print("\nComparison of XGBoost Models:")
print("Initial XGBoost Model:")
print(f"  MSE: {mse_xgb}, MAE: {mae_xgb}, R^2: {r2_xgb}")
print("Optimized XGBoost Model:")
print(f"  MSE: {mse_optimized_xgb}, MAE: {mae_optimized_xgb}, R^2: {r2_optimized_xgb}")



Comparison of XGBoost Models:
Initial XGBoost Model:
  MSE: 952030990.7683752, MAE: 19811.910878638697, R^2: 0.8758813142776489
Optimized XGBoost Model:
  MSE: 736225400.6101568, MAE: 16870.23865582192, R^2: 0.9040164351463318


In [35]:
# Initialize and train the Random Forest regression model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Predict on test set
y_pred_rf = rf_model.predict(X_test)

# Print all evaluation metrics
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Initial Random Forest Model - Mean Squared Error (MSE): {mse_rf}")
print(f"Initial Random Forest Model - Mean Absolute Error (MAE): {mae_rf}")
print(f"Initial Random Forest Model - R^2 Score: {r2_rf}")

# Cross-validation for baseline comparison
cv_scores_rf = cross_val_score(rf_model, X, y, cv=5, scoring='neg_mean_squared_error')
print(f"Cross-Validated MSE (Initial Random Forest Model): {-cv_scores_rf.mean()}")

Initial Random Forest Model - Mean Squared Error (MSE): 993846687.5210072
Initial Random Forest Model - Mean Absolute Error (MAE): 18661.105171232877
Initial Random Forest Model - R^2 Score: 0.8704296713496006
Cross-Validated MSE (Initial Random Forest Model): 891076516.7338301


In [37]:
from skopt import BayesSearchCV

# Define parameter space for Bayesian optimization
param_space_rf = {
    'n_estimators': (100, 1000),  # Number of trees in the forest
    'max_depth': (3, 10),          # Maximum depth of the tree
    'min_samples_split': (2, 20),  # Minimum number of samples required to split an internal node
    'min_samples_leaf': (1, 20)    # Minimum number of samples required to be at a leaf node
}

# Bayesian Optimization with Cross-Validation for Random Forest
bayes_opt_rf = BayesSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    search_spaces=param_space_rf,
    n_iter=10,  # Number of iterations for the search
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42
)

# Fit the Bayesian Optimization
bayes_opt_rf.fit(X_train, y_train)

BayesSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42), n_iter=10,
              random_state=42, scoring='neg_mean_squared_error',
              search_spaces={'max_depth': (3, 10), 'min_samples_leaf': (1, 20),
                             'min_samples_split': (2, 20),
                             'n_estimators': (100, 1000)})

In [38]:
# Retrieve and print the best parameters and best score from Bayesian optimization
print("Best Parameters from Bayesian Optimization (Random Forest):", bayes_opt_rf.best_params_)
print("Best Cross-Validated MSE from Bayesian Optimization (Random Forest):", -bayes_opt_rf.best_score_)

# Using the best estimator from Bayesian Optimization
optimized_rf_model = bayes_opt_rf.best_estimator_

# Predict with the optimized model
y_pred_optimized_rf = optimized_rf_model.predict(X_test)


Best Parameters from Bayesian Optimization (Random Forest): OrderedDict({'max_depth': 9, 'min_samples_leaf': 4, 'min_samples_split': 13, 'n_estimators': 822})
Best Cross-Validated MSE from Bayesian Optimization (Random Forest): 966255488.7945954


In [39]:
# Calculate metrics for optimized model
mse_optimized_rf = mean_squared_error(y_test, y_pred_optimized_rf)
mae_optimized_rf = mean_absolute_error(y_test, y_pred_optimized_rf)
r2_optimized_rf = r2_score(y_test, y_pred_optimized_rf)

# Print metrics for comparison
print("\nComparison of Random Forest Models:")
print("Initial Random Forest Model:")
print(f"  MSE: {mse_rf}, MAE: {mae_rf}, R^2: {r2_rf}")
print("Optimized Random Forest Model:")
print(f"  MSE: {mse_optimized_rf}, MAE: {mae_optimized_rf}, R^2: {r2_optimized_rf}")


Comparison of Random Forest Models:
Initial Random Forest Model:
  MSE: 993846687.5210072, MAE: 18661.105171232877, R^2: 0.8704296713496006
Optimized Random Forest Model:
  MSE: 1060269482.5692017, MAE: 19271.075240000784, R^2: 0.8617699620681418
